#### <font color="#03997A"> Diplomado en Ciencia de datos UNAM </font>

##### <font color="#03997A"> MI: Luis Enrique Argota V.  </font>

##### <font color="#03997A"> Modulo 9 Natural Language Procesing </font>

##### <font color="#03997A"> Tarea 4 - Apredizaje Supervisado para Clasificación de Textos
##### <font color="#03997A"> Equipo 1:
- Sergio Ibarra Ramírez
- Yafte Aaron Flores Morales
- Esteban Bracamontes Orozco

</font>

In [1]:
import re
import pandas as pd
import numpy as np
import nltk
from nltk import *

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

from sklearn.preprocessing import MinMaxScaler

### Leyendo el corpus A

In [2]:
corpus_train_esA = pd.read_csv('train_es.tsv',delimiter='\t',encoding='utf-8')
corpus_dev_esA = pd.read_csv('dev_es.tsv',delimiter='\t',encoding='utf-8')

In [3]:
corpus_train_esA

,id,text,HS,TR,AG
0,20001,Easyjet quiere duplicar el número de mujeres p...,1,0,0
1,20002,El gobierno debe crear un control estricto de ...,1,0,0
2,20003,Yo veo a mujeres destruidas por acoso laboral ...,0,0,0
3,20004,"— Yo soy respetuoso con los demás, sólamente l...",0,0,0
4,20007,Antonio Caballero y como ser de mal gusto e ig...,0,0,0
...,...,...,...,...,...
4464,24996,@miriaan_ac @Linaveso_2105 @HumildesSquad_ CÁL...,1,1,1
4465,24997,"@IvanDuque presidente en Cúcuta , tenemos prob...",1,0,1
4466,24998,- Callaté Visto Que Te Dejo En Puta🎤🎶,0,0,0
4467,24999,-¿porque los hombres se casan con las mujeres?...,1,0,0


### Limpieza en los datos
* Cambiar todas las palabras de mayúsculas a minúsculas
* Se han eliminado las '@' de @USUARIO con el fin de facilitar el etiquetado morfológico
* Quitar los links
* Quitar los emojis
* Eliminar las stopwords
* Se han reemplazado todos los números por el símbolo '0'
* Quitar los signos de puntuación y quitar espacios (tabuladores, etc)

In [4]:
pattern_URL="(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})"

def procesar(file, namefile):
    file[file.columns[1]] = [clean_text(i) for i in file[file.columns[1]]]
    file.to_csv(namefile, sep='\t', encoding='utf-8', index=False)
    return file

def clean_text(text):
    text = text.lower()
    #text=re.sub("@([A-Za-z0-9_]{1,15})", "@USUARIO", text)
    text=re.sub("@([A-Za-z0-9_]{1,15})", " ", text)
    text=re.sub(pattern_URL, " ", text)

    text= remove_emoji(text)
    text= remove_stopwords(text)
    text=re.sub("\d+", "0", text)
    # text=re.sub("\d+", " ", text)

    text=re.sub(r" +", " ", re.sub(r"\t", " ", re.sub(r"\n+", "\n", re.sub('(?:[.,\/!$%?¿?!¡\^&\*;:{}=><\-_`~()”“"\'\|])', " ",text))))
    text = text.strip()
    return text

def remove_stopwords(text):
    stopwords=set(nltk.corpus.stopwords.words("spanish"))
    for i in stopwords:
        text = re.sub(r"\b%s\b" % i, " ", text)
    return text

def remove_emoji(text):
    emoji_pattern = re.compile("["
                               "\U0001F600-\U0001F64F"  # emoticons
                               "\U0001F300-\U0001F5FF"  # symbols & pictographs
                               "\U0001F680-\U0001F6FF"  # transport & map symbols
                               "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "\U00002702-\U000027B0"
                               "\U000024C2-\U0001F251"
                               "\U0001f926-\U0001f937"
                               "\u200d"
                               "\u2640-\u2642"
                               "\U0001F1F2-\U0001F1F4"  # Macau flag
                               "\U0001F1E6-\U0001F1FF"  # flags
                               "\U0001F600-\U0001F64F"
                               "\U0001F1F2"
                               "\U0001F1F4"
                               "\U0001F620"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) # no emoji
    return text

### Guardando el corpus ya procesado A

In [6]:
nltk.download('stopwords')
corpus_train_esA = procesar(corpus_train_esA, "train_es_cleanA.tsv")
corpus_dev_esA = procesar(corpus_dev_esA, "dev_es_cleanA.tsv")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sergio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
#Leyendo el corpus ya procesado-limpio A

train_idA = corpus_train_esA[corpus_train_esA.columns[0]]
X_train_textA = corpus_train_esA[corpus_train_esA.columns[1]].fillna(' ')
y_train_hsA = corpus_train_esA[corpus_train_esA.columns[2]]

test_idA = corpus_dev_esA[corpus_train_esA.columns[0]]
X_test_textA = corpus_dev_esA[corpus_dev_esA.columns[1]].fillna(' ')
y_test_hsA = corpus_dev_esA[corpus_dev_esA.columns[2]]

In [8]:
print( len(X_train_textA), len(y_train_hsA) )

print( len(X_test_textA), len(y_test_hsA) )

4469 4469
500 500


In [9]:
cvectorizer = CountVectorizer(
    # lowercase=True,
    #stop_words=[word.decode('utf-8') for word in nltk.corpus.stopwords.words('spanish')],
    #token_pattern=r'\b\w+\b', #selects tokens of 2 or more alphanumeric characters
    ngram_range=(1,3),#n-grams de palabras n = 1 a n = 3 (unigramas, bigramas y trigramas)
    min_df=5,#ignorando los términos que tienen una frecuencia de documento estrictamente inferior a 5
).fit(X_train_textA)

X_train_cvectorized = cvectorizer.transform(X_train_textA).toarray()
print(X_train_cvectorized.shape)

X_test_cvectorized = cvectorizer.transform(X_test_textA).toarray()
print(X_test_cvectorized.shape)

(4469, 1775)
(500, 1775)


In [10]:
tvectorizer = TfidfVectorizer(
    # lowercase=True,
    #stop_words=[word.decode('utf-8') for word in nltk.corpus.stopwords.words('spanish')],
    #token_pattern=r'\b\w+\b', #selects tokens of 2 or more alphanumeric characters
    ngram_range=(1,3),#n-grams de palabras n = 1 a n = 3 (unigramas, bigramas y trigramas)
    min_df=5,#ignorando los términos que tienen una frecuencia de documento estrictamente inferior a 5
).fit(X_train_textA)

X_train_tvectorized = tvectorizer.transform(X_train_textA).toarray()
print(X_train_tvectorized.shape)

X_test_tvectorized = tvectorizer.transform(X_test_textA).toarray()
print(X_test_tvectorized.shape)

(4469, 1775)
(500, 1775)


In [11]:
X_train_tvectorized[:1]

array([[0., 0., 0., ..., 0., 0., 0.]])

### (1) Ejecución de Modelo Multinomial

In [12]:
vect = CountVectorizer().fit(X_train_textA)
X_train_vectorized = vect.transform(X_train_textA)

In [14]:
clf1 = MultinomialNB()
clf1.fit(X_train_vectorized, y_train_hsA) # ajusta al calificador 1

MultinomialNB()

In [15]:
X_test_vectorized = vect.transform(X_test_textA)
predictions1 = clf1.predict(X_test_vectorized) # funcion para predecir 1

In [16]:
print('Predicciones y resultados del clasificador 1')
print('Predicciones', predictions1)
# métricas para verificar que tanto es el calificador es correcto
print( 'Accuracy', accuracy_score(y_test_hsA, predictions1) )
print( 'AUC', roc_auc_score(y_test_hsA, predictions1) )
print( 'F1', f1_score(y_test_hsA, predictions1) )

Predicciones y resultados del clasificador 1
Predicciones [0 1 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 1 1 0 1 0 0
 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 1 1 0 0 0 1 1 0
 1 1 0 0 0 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0 0 0 0 1 0 1 1 0 1 0 1 1 1 0 0 0 1
 0 1 0 0 1 1 0 1 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0
 0 0 0 0 1 1 1 0 0 1 0 0 0 1 1 0 0 0 1 0 1 0 1 0 0 1 0 1 1 0 1 1 0 1 1 0 0
 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 1 0 1 0
 1 0 0 1 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0
 1 1 0 0 0 1 0 1 0 1 1 1 0 1 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0
 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1
 0 0 0 1 0 1 0 1 0 0 0 1 1 0 1 1 1 0 1 0 1 1 1 1 0 1 0 1 1 0 0 0 1 0 0 0 1
 1 0 0 0 0 0 0 0 1 0 1 1 0 1 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
 1 0 0 1 1 0 0 1 1 0 0 0 1 1 1 1 0 0 1 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 1 0 0
 1 1 1 1 0 0 1 0 1 1 1 0 0 0 0 0 0 1 1 0 1

### (2) Ejecución de Modelo SVC

In [17]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [18]:
from sklearn.svm import SVC, LinearSVC

vect = TfidfVectorizer(min_df=5).fit(X_train_textA)
X_train_vectorized = vect.transform(X_train_textA)

num_chars = X_train_textA.str.len()

print(X_train_vectorized.shape)
X_train_vectorized = add_feature(X_train_vectorized, num_chars)
print(X_train_vectorized.shape)

model = SVC(C=100) # con regularización 'C = 10000'
modelLin = LinearSVC()

model.fit(X_train_vectorized, y_train_hsA)
modelLin.fit(X_train_vectorized, y_train_hsA)

X_test_vectorized = vect.transform(X_test_textA)
num_charsTest = X_test_textA.str.len()
X_test_vectorized = add_feature(X_test_vectorized, num_charsTest)

predictions = model.predict(X_test_vectorized)
predictionsLin = modelLin.predict(X_test_vectorized)

print(f1_score(y_test_hsA, predictions), f1_score(y_test_hsA, predictionsLin))

(4469, 1527)
(4469, 1528)


c:\Users\Sergio\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.3310344827586207 0.719298245614035


### (3) Ejecución de Modelo Árboles de Clasificación

In [19]:
from sklearn.tree import DecisionTreeClassifier

In [20]:
vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train_textA)
X_train_vectorized = vect.transform(X_train_textA)

num_chars = X_train_textA.str.len()
num_digits = X_train_textA.str.count('\d')

X_train_vectorized = add_feature(X_train_vectorized, num_chars)
X_train_vectorized = add_feature(X_train_vectorized, num_digits)

In [21]:
arbol = DecisionTreeClassifier(max_depth=2)
arbol.fit(X_train_vectorized,y_train_hsA)

DecisionTreeClassifier(max_depth=2)

In [22]:
X_test_vectorized = vect.transform(X_test_textA)
num_charsTest = X_test_textA.str.len()
num_digitsTest = X_test_textA.str.count('\d')
X_test_vectorized = add_feature(X_test_vectorized, num_charsTest)
X_test_vectorized = add_feature(X_test_vectorized, num_digitsTest)

In [23]:
y_pred_hsA = arbol.predict(X_test_vectorized)

In [24]:
from sklearn.metrics import classification_report

print(classification_report(y_test_hsA,y_pred_hsA))

              precision    recall  f1-score   support

           0       0.63      0.93      0.75       278
           1       0.78      0.32      0.46       222

    accuracy                           0.66       500
   macro avg       0.71      0.63      0.61       500
weighted avg       0.70      0.66      0.62       500



### (4) Ejecución de Modelo KNN

In [25]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier

In [26]:
vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train_textA)
X_train_vectorized = vect.transform(X_train_textA)

num_chars = X_train_textA.str.len()
num_digits = X_train_textA.str.count('\d')

X_train_vectorized = add_feature(X_train_vectorized, num_chars)
X_train_vectorized = add_feature(X_train_vectorized, num_digits)

In [27]:
modelo = KNeighborsClassifier(n_neighbors=5)
modelo.fit(X_train_vectorized,y_train_hsA)

KNeighborsClassifier()

In [28]:
X_test_vectorized = vect.transform(X_test_textA)
num_charsTest = X_test_textA.str.len()
num_digitsTest = X_test_textA.str.count('\d')
X_test_vectorized = add_feature(X_test_vectorized, num_charsTest)
X_test_vectorized = add_feature(X_test_vectorized, num_digitsTest)

In [29]:
y_pred_hsA = arbol.predict(X_test_vectorized)

In [30]:
from sklearn.metrics import classification_report

print(classification_report(y_test_hsA,y_pred_hsA))

              precision    recall  f1-score   support

           0       0.63      0.93      0.75       278
           1       0.78      0.32      0.46       222

    accuracy                           0.66       500
   macro avg       0.71      0.63      0.61       500
weighted avg       0.70      0.66      0.62       500



### (5) Ejecución de Modelo Logístico

In [31]:
from sklearn.linear_model import LogisticRegression

vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train_textA)
X_train_vectorized = vect.transform(X_train_textA)

num_chars = X_train_textA.str.len()
num_digits = X_train_textA.str.count('\d')

X_train_vectorized = add_feature(X_train_vectorized, num_chars)
X_train_vectorized = add_feature(X_train_vectorized, num_digits)

model = LogisticRegression(C=100)
model.fit(X_train_vectorized, y_train_hsA)

X_test_vectorized = vect.transform(X_test_textA)
num_charsTest = X_test_textA.str.len()
num_digitsTest = X_test_textA.str.count('\d')
X_test_vectorized = add_feature(X_test_vectorized, num_charsTest)
X_test_vectorized = add_feature(X_test_vectorized, num_digitsTest)

predictions = model.predict(X_test_vectorized)
print(f1_score(y_test_hsA, predictions))

0.761904761904762


c:\Users\Sergio\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
